# Transition frequency calculator
Calculating transition frequencies between given states at zero electric and magnetic field.

## Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

import copy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy

from centrex_TlF_hamiltonian.states import State, CoupledBasisState, UncoupledBasisState, generate_uncoupled_states_ground_mF0, generate_coupled_states_excited_mF0, find_state_idx_from_state, ElectronicState
from centrex_TlF_hamiltonian.hamiltonian import generate_uncoupled_hamiltonian_X, generate_uncoupled_hamiltonian_X_function, generate_coupled_hamiltonian_B, generate_coupled_hamiltonian_B_function, XConstants, BConstants
from centrex_TlF_hamiltonian.hamiltonian.utils import matrix_to_states

## Tl-205
### X-state Hamiltonian

In [ ]:
# Generate quantum numbers
QN_X_uc = generate_uncoupled_states_ground_mF0(range(0,10))

# Generate Hamiltonian
H_X = generate_uncoupled_hamiltonian_X_function(generate_uncoupled_hamiltonian_X(QN_X_uc))(np.array([0,0,0]),np.array([0,0,1e-4]))/(2*np.pi)

#Diagonalize the Hamiltonian and define a new basis where the matrix is diagonal
D, V = np.linalg.eigh(H_X)
H_X_diag = V.conj().T @ H_X @ V

#Remove very small entries from V
V[np.abs(V) < 1e-5] = 0

#New set of quantum numbers:
QN_X_diag = matrix_to_states(V, QN_X_uc)

### B-state Hamiltonian

In [ ]:
# Generate quantum numbers
QN_B_c = generate_coupled_states_excited_mF0(range(1,11), Ps = [-1,1])

# Generate Hamiltonian
H_B = generate_coupled_hamiltonian_B_function(generate_coupled_hamiltonian_B(QN_B_c))(np.array([0,0,0]),np.array([0,0,1e-4]))/(2*np.pi)

#Diagonalize the Hamiltonian and define a new basis where the matrix is diagonal
D, V = np.linalg.eigh(H_B)
H_B_diag = V.conj().T @ H_B @ V

#Remove very small entries from V
V[np.abs(V) < 1e-5] = 0

#New set of quantum numbers:
QN_B_diag = matrix_to_states(V, QN_B_c)

### Merge Hamiltonians
Merge the X- and B-state Hamiltonians and add and offset to the B-state energies to include the optical wavelength

In [ ]:
#Add an offset to the energies of the B-states so that the transition frequency from |X, J = 0, F=1> to |B, J = 1, F = 2> is 
#1103407.96 GHz
X_state = 1*CoupledBasisState(J=0, F1 = 1/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, electronic_state=ElectronicState.X, P = +1, Omega = 0)
B_state = 1*CoupledBasisState(J=1, F1 = 3/2, F = 2, mF = 0, I1 = 1/2, I2 = 1/2, electronic_state=ElectronicState.B, P = -1, Omega = 1)

#Find the indices that correspond to each state
i_X = find_state_idx_from_state(H_X_diag, X_state, QN_X_diag)
i_B = find_state_idx_from_state(H_B_diag, B_state, QN_B_diag)

#Find the correct energy shift
E_shift = 4*275851.99e9- H_B_diag[i_B,i_B] + H_X_diag[i_X,i_X] 

#Shift the energies of the B-states
H_B_diag = H_B_diag + E_shift*np.eye(H_B_diag.shape[0])

# Make the total Hamiltonian for Tl-205
H_tot = scipy.linalg.block_diag(H_X_diag, H_B_diag)
QN = QN_X_diag + QN_B_diag

## Tl-203
### X-state Hamiltonian

In [ ]:
# Generate quantum numbers
QN_X_uc = generate_uncoupled_states_ground_mF0(range(0,10))

# Generate Hamiltonian
X_constants_203 = copy.deepcopy(XConstants)
X_constants_203.B_rot *= 1.00084 
H_X_203 = generate_uncoupled_hamiltonian_X_function(generate_uncoupled_hamiltonian_X(QN_X_uc, X_constants_203))(np.array([0,0,0]),np.array([0,0,1e-4]))/(2*np.pi)

#Diagonalize the Hamiltonian and define a new basis where the matrix is diagonal
D, V = np.linalg.eigh(H_X_203)
H_X_203_diag = V.conj().T @ H_X_203 @ V

#Remove very small entries from V
V[np.abs(V) < 1e-5] = 0

#New set of quantum numbers:
QN_X_203_diag = matrix_to_states(V, QN_X_uc)

### B-state Hamiltonian

In [ ]:
# Generate quantum numbers
QN_B_c = generate_coupled_states_excited_mF0(range(1,11), Ps = [-1,1])

# Generate Hamiltonian
B_constants_203 = copy.deepcopy(BConstants)
B_constants_203.B_rot = 6694.42e6
B_constants_203.h1_Tl = 28516e6
B_constants_203.h1_F = 864e6
B_constants_203.q = 2.423e6
B_constants_203.c_Tl = -13.9e6
B_constants_203.c1p_Tl = 11.17e6

H_B_203 = generate_coupled_hamiltonian_B_function(generate_coupled_hamiltonian_B(QN_B_c, B_constants_203))(np.array([0,0,0]),np.array([0,0,1e-4]))/(2*np.pi)

#Diagonalize the Hamiltonian and define a new basis where the matrix is diagonal
D, V = np.linalg.eigh(H_B)
H_B_203_diag = V.conj().T @ H_B_203 @ V

#Remove very small entries from V
V[np.abs(V) < 1e-5] = 0

#New set of quantum numbers:
QN_B_203_diag = matrix_to_states(V, QN_B_c)

### Merge Hamiltonians
Merge the X- and B-state Hamiltonians and add and offset to the B-state energies to include the optical wavelength

In [ ]:
#Add an offset to the energies of the B-states so that the transition frequency from |X, J = 3, F=4> to |B, J = 4, F = 5> is 
#4*275862.82 GHz
X_state = 1*CoupledBasisState(J=3, F1 = 7/2, F = 4, mF = 0, I1 =1/2, I2 = 1/2, electronic_state=ElectronicState.X, P = -1, Omega = 0)
B_state = 1*CoupledBasisState(J=4, F1 = 9/2, F = 5, mF = 0, I1 =1/2, I2 = 1/2, electronic_state=ElectronicState.B, P = +1, Omega = 1)

#Find the indices that correspond to each state
i_X_203 = find_state_idx_from_state(H_X_203_diag, X_state, QN_X_203_diag)
i_B_203 = find_state_idx_from_state(H_B_203_diag, B_state, QN_B_203_diag)

#Find the correct energy shift
E_shift = 4*275862.82e9 - H_B_203_diag[i_B_203,i_B_203] + H_X_203_diag[i_X_203,i_X_203] 

#Shift the energies of the B-states
H_B_203_diag = H_B_203_diag + E_shift*np.eye(H_B_203_diag.shape[0])


H_tot_203 = scipy.linalg.block_diag(H_X_203_diag, H_B_203_diag)
QN_203 = QN_X_203_diag + QN_B_203_diag

## Calculate transition frequencies
### Function for calculations

In [ ]:
def calculate_transition_frequency(state1, state2, H, QN):
    #Find indices corresponding to each state
    i1 = find_state_idx_from_state(H, state1, QN)
    i2 = find_state_idx_from_state(H, state2, QN)
    
#     print(i1)
#     print(i2)
    
    #Find energies of each state
    E1 = H[i1,i1]
    E2 = H[i2,i2]
    
    #Calculate transition frequency
    f = E2 - E1
    
    return np.real(f)

### Check some transition frequencies
Checking that frequencies of some R-branch transitions roughly match with experimental results 

In [ ]:
R0_X = 1*CoupledBasisState(J=0, F1 = 1/2, F = 1, mF = 0, I1 =1/2, I2 = 1/2, electronic_state=ElectronicState.X, P = +1, Omega = 0)
R0_B = 1*CoupledBasisState(J=1, F1 = 3/2, F = 2, mF = 0, I1 =1/2, I2 = 1/2, electronic_state=ElectronicState.B, P = -1, Omega = 1)

calculate_transition_frequency(R0_X, R0_B, H_tot, QN)/(1e9*4)

In [ ]:
R1_X = 1*CoupledBasisState(J=1, F1 = 3/2, F = 2, mF = 0, I1 =1/2, I2 = 1/2, electronic_state=ElectronicState.X, P = -1, Omega = 0)
R1_B = 1*CoupledBasisState(J=2, F1 = 5/2, F = 3, mF = 0, I1 =1/2, I2 = 1/2, electronic_state=ElectronicState.B, P = +1, Omega = 1)

calculate_transition_frequency(R1_X, R1_B, H_tot, QN)/(1e9*4)

In [ ]:
R2_X = 1*CoupledBasisState(J=2, F1 = 5/2, F = 3, mF = 0, I1 =1/2, I2 = 1/2, electronic_state=ElectronicState.X, P = +1, Omega = 0)
R2_B = 1*CoupledBasisState(J=3, F1 = 5/2, F = 2, mF = 0, I1 =1/2, I2 = 1/2, electronic_state=ElectronicState.B, P = -1, Omega = 1)

calculate_transition_frequency(R2_X, R2_B, H_tot, QN)/(1e9*4)

In [ ]:
R3_X = 1*CoupledBasisState(J=3, F1 = 7/2, F = 4, mF = 0, I1 =1/2, I2 = 1/2, electronic_state=ElectronicState.X, P = -1, Omega = 0)
R3_B = 1*CoupledBasisState(J=4, F1 = 7/2, F = 4, mF = 0, I1 =1/2, I2 = 1/2, electronic_state=ElectronicState.B, P = +1, Omega = 1)

calculate_transition_frequency(R3_X, R3_B, H_tot, QN)/(1e9*4)

In [ ]:
X_state = 1*CoupledBasisState(J=3, F1 = 7/2, F = 4, mF = 0, I1 =1/2, I2 = 1/2, electronic_state=ElectronicState.X, P = -1, Omega = 0)
B_state = 1*CoupledBasisState(J=4, F1 = 9/2, F = 5, mF = 0, I1 =1/2, I2 = 1/2, electronic_state=ElectronicState.B, P = +1, Omega = 1)

calculate_transition_frequency(X_state, B_state, H_tot_203, QN_203)/(1e9*4)

### Q-branch frequencies
Calculating frequencies for all Q-branch transitions with J = 1,2,3,4,5

In [ ]:
Js = [1,2,3,4,5]
I1 = 1/2
I2 = 1/2
#Generate all excited states
B_states = []
for J in Js:
    for F1 in np.arange(J-I1, J+I1+1):
        for F in np.arange(F1-I1, F1+I1+1):
            state = 1*CoupledBasisState(J=J, F1 = F1, F = F, mF = 0, I1 =I1, I2 = I2, electronic_state=ElectronicState.B, 
                                        P = (-1)**(J+1), Omega = 1)
            B_states.append(state)

#Generate ground states(only one for each J since they are almost degenerate)
X_states = []
for J in Js:
    F1 = J+1/2
    F = J
    state = 1*CoupledBasisState(J=J, F1 = F1, F = F, mF = 0, I1 =I1, I2 = I2, electronic_state=ElectronicState.X, 
                                P = (-1)**J, Omega = 0)
    X_states.append(state)

In [ ]:
#Calculate transition frequencies for each J for X and each hyperfine state for B
Q_transitions = []
Q_frequencies = []

for J in Js:
    print(10*'*')
    #Determine X-state
    F1_X = J+1/2
    F_X = J
    X_state = 1*CoupledBasisState(J=J, F1 = F1_X, F = F_X, mF = 0, I1 =I1, I2 = I2, electronic_state=ElectronicState.X, 
                                P = (-1)**J, Omega = 0)
    
    #Determine B-states
    for F1 in np.arange(J-I1, J+I1+1):
        for F in np.arange(F1-I1, F1+I1+1):
            B_state = 1*CoupledBasisState(J=J, F1 = F1, F = F, mF = 0, I1 = I1, I2 = I2, electronic_state=ElectronicState.B, 
                                        P = (-1)**(J+1), Omega = 1)
            
            #Calculate transition frequency
            frq = calculate_transition_frequency(X_state, B_state, H_tot, QN)/(1e9*4) #In GHz at IR
            
            #Print
            print(f"Frequency for Q{J} F1' = {F1}, F' = {F} is f = {frq}  GHz")
            Q_transitions.append(f"Q{J} F1' = {F1}, F' = {F}")
            Q_frequencies.append(frq)

### R-branch frequencies
Loop over each J and calculate transition frequencies for R-branch transitions. Only using one state for the ground state since they are basically degenerate

In [ ]:
R_transitions = []
R_frequencies = []

#Calculate transition frequencies for each J for X and each hyperfine state for B
Js = [0,1,2,3,4,5,6,7,8,9]
for J_X in Js:
    print(10*'*')
    #Determine X-state
    F1_X = J_X+1/2
    F_X = J_X
    X_state = 1*CoupledBasisState(J=J_X, F1 = F1_X, F = F_X, mF = 0, I1 =I1, I2 = I2, electronic_state=ElectronicState.X, 
                                P = (-1)**J_X, Omega = 0)
    
    #Determine B-states
    J_B = J_X+1
    for F1 in np.arange(J_B-I1, J_B+I1+1):
        for F in np.arange(F1-I2, F1+I2+1):
            B_state = 1*CoupledBasisState(J=J_B, F1 = F1, F = F, mF = 0, I1 = I1, I2 = I2, electronic_state=ElectronicState.B, 
                                        P = (-1)**(J_X+1), Omega = 1)
            
            #Calculate transition frequency
            frq = calculate_transition_frequency(X_state, B_state, H_tot, QN)/(1e9*4) #In GHz at IR
            
            #Print
            print(f"Frequency for R{J_X} F1' = {F1}, F' = {F} is f = {frq}  GHz")
            
            R_transitions.append(f"R{J_X} F1' = {F1}, F' = {F}")
            R_frequencies.append(frq)

## P-branch frequencies
Loop over each J and calculate transition frequencies for R-branch transitions. Only using one state for the ground state since they are basically degenerate

In [ ]:
P_transitions = []
P_frequencies = []

#Calculate transition frequencies for each J for X and each hyperfine state for B
Js = [2,3,4,5,6,7,8,9]
for J_X in Js:
    print(10*'*')
    #Determine X-state
    F1_X = J_X+1/2
    F_X = J_X
    X_state = 1*CoupledBasisState(J=J_X, F1 = F1_X, F = F_X, mF = 0, I1 =I1, I2 = I2, electronic_state=ElectronicState.X, 
                                P = (-1)**J_X, Omega = 0)
    
    #Determine B-states
    J_B = J_X-1
    for F1 in np.arange(J_B-I1, J_B+I1+1):
        for F in np.arange(F1-I2, F1+I2+1):
            B_state = 1*CoupledBasisState(J=J_B, F1 = F1, F = F, mF = 0, I1 = I1, I2 = I2, electronic_state=ElectronicState.B, 
                                        P = (-1)**(J_X+1), Omega = 1)
            
            #Calculate transition frequency
            frq = calculate_transition_frequency(X_state, B_state, H_tot, QN)/(1e9*4) #In GHz at IR
            
            #Print
            print(f"Frequency for P{J_X} F1' = {F1}, F' = {F} is f = {frq}  GHz")
            
            P_transitions.append(f"P{J_X} F1' = {F1}, F' = {F}")
            P_frequencies.append(frq)

### S-branch transitions:

In [ ]:
S_transitions = []
S_frequencies = []

#Calculate transition frequencies for each J for X and each hyperfine state for B
Js = [0,1,2,3,4,5,6,7,8]
for J_X in Js:
    print(10*'*')
    #Determine X-state
    F1_X = J_X+1/2
    F_X = J_X
    X_state = 1*CoupledBasisState(J=J_X, F1 = F1_X, F = F_X, mF = 0, I1 =I1, I2 = I2, electronic_state=ElectronicState.X, 
                                P = (-1)**J_X, Omega = 0)
    
    #Determine B-states
    J_B = J_X+2
    for F1 in np.arange(J_B-I1, J_B+I1+1):
        for F in np.arange(F1-I2, F1+I2+1):
            B_state = 1*CoupledBasisState(J=J_B, F1 = F1, F = F, mF = 0, I1 = I1, I2 = I2, electronic_state=ElectronicState.B, 
                                        P = (-1)**(J_X+1), Omega = 1)
            
            #Calculate transition frequency
            frq = calculate_transition_frequency(X_state, B_state, H_tot, QN)/(1e9*4) #In GHz at IR
            
            #Print
            print(f"Frequency for S{J_X} F1' = {F1}, F' = {F} is f = {frq} GHz")
            S_transitions.append(f"S{J_X} F1' = {F1}, F' = {F}")
            S_frequencies.append(frq)

### O-branch transitions:


In [ ]:
O_transitions = []
O_frequencies = []

#Calculate transition frequencies for each J for X and each hyperfine state for B
Js = [3,4,5,6,7,8]
for J_X in Js:
    print(10*'*')
    #Determine X-state
    F1_X = J_X+1/2
    F_X = J_X
    X_state = 1*CoupledBasisState(J=J_X, F1 = F1_X, F = F_X, mF = 0, I1 =I1, I2 = I2, electronic_state=ElectronicState.X, 
                                P = (-1)**J_X, Omega = 0)
    
    #Determine B-states
    J_B = J_X-2
    for F1 in np.arange(J_B-I1, J_B+I1+1):
        for F in np.arange(F1-I2, F1+I2+1):
            B_state = 1*CoupledBasisState(J=J_B, F1 = F1, F = F, mF = 0, I1 = I1, I2 = I2, electronic_state=ElectronicState.B, 
                                        P = (-1)**(J_X+1), Omega = 1)
            
            #Calculate transition frequency
            frq = calculate_transition_frequency(X_state, B_state, H_tot, QN)/(1e9*4) #In GHz at IR
            
            #Print
            print(f"Frequency for O{J_X} F1' = {F1}, F' = {F} is f = {frq} GHz")
            O_transitions.append(f"S{J_X} F1' = {F1}, F' = {F}")
            O_frequencies.append(frq)

### T-branch transitions:

In [ ]:
T_transitions = []
T_frequencies = []

#Calculate transition frequencies for each J for X and each hyperfine state for B
Js = [0,1,2,3,4,5,6]
for J_X in Js:
    print(10*'*')
    #Determine X-state
    F1_X = J_X+1/2
    F_X = J_X
    X_state = 1*CoupledBasisState(J=J_X, F1 = F1_X, F = F_X, mF = 0, I1 =I1, I2 = I2, electronic_state=ElectronicState.X, 
                                P = (-1)**J_X, Omega = 0)
    
    #Determine B-states
    J_B = J_X+3
    for F1 in np.arange(J_B-I1, J_B+I1+1):
        for F in np.arange(F1-I2, F1+I2+1):
            B_state = 1*CoupledBasisState(J=J_B, F1 = F1, F = F, mF = 0, I1 = I1, I2 = I2, electronic_state=ElectronicState.B, 
                                        P = (-1)**(J_X+1), Omega = 1)
            
            #Calculate transition frequency
            frq = calculate_transition_frequency(X_state, B_state, H_tot, QN)/(1e9*4) #In GHz at IR
            
            #Print
            print(f"Frequency for T{J_X} F1' = {F1}, F' = {F} is f = {frq} GHz")
            T_transitions.append(f"T{J_X} F1' = {F1}, F' = {F}")
            T_frequencies.append(frq)

## Frequencies for Tl-203

In [ ]:
## Q-branch frequencies
# Calculating frequencies for all Q-branch transitions with J = 1,2,3

Js = [1,2,3,4,5]
I1 = 1/2
I2 = 1/2
#Generate all excited states
B_states = []
for J in Js:
    for F1 in np.arange(J-I1, J+I1+1):
        for F in np.arange(F1-I1, F1+I1+1):
            state = 1*CoupledBasisState(J=J, F1 = F1, F = F, mF = 0, I1 =I1, I2 = I2, electronic_state=ElectronicState.B, 
                                        P = (-1)**(J+1), Omega = 1)
            B_states.append(state)

#Generate ground states(only one for each J since they are almost degenerate)
X_states = []
for J in Js:
    F1 = J+1/2
    F = J
    state = 1*CoupledBasisState(J=J, F1 = F1, F = F, mF = 0, I1 =I1, I2 = I2, electronic_state=ElectronicState.X, 
                                P = (-1)**J, Omega = 0)
    X_states.append(state)

#Calculate transition frequencies for each J for X and each hyperfine state for B
Q_transitions_203 = []
Q_frequencies_203 = []

for J in Js:
    print(10*'*')
    #Determine X-state
    F1_X = J+1/2
    F_X = J
    X_state = 1*CoupledBasisState(J=J, F1 = F1_X, F = F_X, mF = 0, I1 =I1, I2 = I2, electronic_state=ElectronicState.X, 
                                P = (-1)**J, Omega = 0)
    
    #Determine B-states
    for F1 in np.arange(J-I1, J+I1+1):
        for F in np.arange(F1-I1, F1+I1+1):
            B_state = 1*CoupledBasisState(J=J, F1 = F1, F = F, mF = 0, I1 = I1, I2 = I2, electronic_state=ElectronicState.B, 
                                        P = (-1)**(J+1), Omega = 1)
            
            #Calculate transition frequency
            frq = calculate_transition_frequency(X_state, B_state, H_tot_203, QN_203)/(1e9*4) #In GHz at IR
            
            #Print
            print(f"Frequency for Q{J} F1' = {F1}, F' = {F} is f = {frq}  GHz")
            Q_transitions_203.append(f"Q{J} F1' = {F1}, F' = {F} (Tl-203)")
            Q_frequencies_203.append(frq)

## R-branch frequencies
# Loop over each J and calculate transition frequencies for R-branch transitions. Only using one state for the ground state since they are basically degenerate

R_transitions_203 = []
R_frequencies_203 = []

#Calculate transition frequencies for each J for X and each hyperfine state for B
Js = [0,1,2,3,4,5,6,7,8,9]
for J_X in Js:
    print(10*'*')
    #Determine X-state
    F1_X = J_X+1/2
    F_X = J_X
    X_state = 1*CoupledBasisState(J=J_X, F1 = F1_X, F = F_X, mF = 0, I1 =I1, I2 = I2, electronic_state=ElectronicState.X, 
                                P = (-1)**J_X, Omega = 0)
    
    #Determine B-states
    J_B = J_X+1
    for F1 in np.arange(J_B-I1, J_B+I1+1):
        for F in np.arange(F1-I2, F1+I2+1):
            B_state = 1*CoupledBasisState(J=J_B, F1 = F1, F = F, mF = 0, I1 = I1, I2 = I2, electronic_state=ElectronicState.B, 
                                        P = (-1)**(J_X+1), Omega = 1)
            
            #Calculate transition frequency
            frq = calculate_transition_frequency(X_state, B_state, H_tot_203, QN_203)/(1e9*4) #In GHz at IR
            
            #Print
            print(f"Frequency for R{J_X} F1' = {F1}, F' = {F} is f = {frq}  GHz")
            
            R_transitions_203.append(f"R{J_X} F1' = {F1}, F' = {F} (Tl-203)")
            R_frequencies_203.append(frq)

## P-branch frequencies
# Loop over each J and calculate transition frequencies for R-branch transitions. Only using one state for the ground state since they are basically degenerate

P_transitions_203 = []
P_frequencies_203 = []

#Calculate transition frequencies for each J for X and each hyperfine state for B
Js = [2,3,4,5,6,7,8,9]
for J_X in Js:
    print(10*'*')
    #Determine X-state
    F1_X = J_X+1/2
    F_X = J_X
    X_state = 1*CoupledBasisState(J=J_X, F1 = F1_X, F = F_X, mF = 0, I1 =I1, I2 = I2, electronic_state=ElectronicState.X, 
                                P = (-1)**J_X, Omega = 0)
    
    #Determine B-states
    J_B = J_X-1
    for F1 in np.arange(J_B-I1, J_B+I1+1):
        for F in np.arange(F1-I2, F1+I2+1):
            B_state = 1*CoupledBasisState(J=J_B, F1 = F1, F = F, mF = 0, I1 = I1, I2 = I2, electronic_state=ElectronicState.B, 
                                        P = (-1)**(J_X+1), Omega = 1)
            
            #Calculate transition frequency
            frq = calculate_transition_frequency(X_state, B_state, H_tot_203, QN_203)/(1e9*4) #In GHz at IR
            
            #Print
            print(f"Frequency for P{J_X} F1' = {F1}, F' = {F} is f = {frq}  GHz")
            
            P_transitions_203.append(f"P{J_X} F1' = {F1}, F' = {F} (Tl-203)")
            P_frequencies_203.append(frq)

## S-branch transitions:

S_transitions_203 = []
S_frequencies_203 = []

#Calculate transition frequencies for each J for X and each hyperfine state for B
Js = [0,1,2,3,4,5,6,7,8]
for J_X in Js:
    print(10*'*')
    #Determine X-state
    F1_X = J_X+1/2
    F_X = J_X
    X_state = 1*CoupledBasisState(J=J_X, F1 = F1_X, F = F_X, mF = 0, I1 =I1, I2 = I2, electronic_state=ElectronicState.X, 
                                P = (-1)**J_X, Omega = 0)
    
    #Determine B-states
    J_B = J_X+2
    for F1 in np.arange(J_B-I1, J_B+I1+1):
        for F in np.arange(F1-I2, F1+I2+1):
            B_state = 1*CoupledBasisState(J=J_B, F1 = F1, F = F, mF = 0, I1 = I1, I2 = I2, electronic_state=ElectronicState.B, 
                                        P = (-1)**(J_X+1), Omega = 1)
            
            #Calculate transition frequency
            frq = calculate_transition_frequency(X_state, B_state, H_tot_203, QN_203)/(1e9*4) #In GHz at IR
            
            #Print
            print(f"Frequency for S{J_X} F1' = {F1}, F' = {F} is f = {frq} GHz")
            S_transitions_203.append(f"S{J_X} F1' = {F1}, F' = {F} (Tl-203)")
            S_frequencies_203.append(frq)

## O-branch transitions:

O_transitions_203 = []
O_frequencies_203 = []

#Calculate transition frequencies for each J for X and each hyperfine state for B
Js = [3,4,5,6,7,8]
for J_X in Js:
    print(10*'*')
    #Determine X-state
    F1_X = J_X+1/2
    F_X = J_X
    X_state = 1*CoupledBasisState(J=J_X, F1 = F1_X, F = F_X, mF = 0, I1 =I1, I2 = I2, electronic_state=ElectronicState.X, 
                                P = (-1)**J_X, Omega = 0)
    
    #Determine B-states
    J_B = J_X-2
    for F1 in np.arange(J_B-I1, J_B+I1+1):
        for F in np.arange(F1-I2, F1+I2+1):
            B_state = 1*CoupledBasisState(J=J_B, F1 = F1, F = F, mF = 0, I1 = I1, I2 = I2, electronic_state=ElectronicState.B, 
                                        P = (-1)**(J_X+1), Omega = 1)
            
            #Calculate transition frequency
            frq = calculate_transition_frequency(X_state, B_state, H_tot_203, QN_203)/(1e9*4) #In GHz at IR
            
            #Print
            print(f"Frequency for O{J_X} F1' = {F1}, F' = {F} is f = {frq} GHz")
            O_transitions_203.append(f"S{J_X} F1' = {F1}, F' = {F} (Tl-203)")
            O_frequencies_203.append(frq)

## T-branch transitions:

T_transitions_203 = []
T_frequencies_203 = []

#Calculate transition frequencies for each J for X and each hyperfine state for B
Js = [0,1,2,3,4,5,6]
for J_X in Js:
    print(10*'*')
    #Determine X-state
    F1_X = J_X+1/2
    F_X = J_X
    X_state = 1*CoupledBasisState(J=J_X, F1 = F1_X, F = F_X, mF = 0, I1 =I1, I2 = I2, electronic_state=ElectronicState.X, 
                                P = (-1)**J_X, Omega = 0)
    
    #Determine B-states
    J_B = J_X+3
    for F1 in np.arange(J_B-I1, J_B+I1+1):
        for F in np.arange(F1-I2, F1+I2+1):
            B_state = 1*CoupledBasisState(J=J_B, F1 = F1, F = F, mF = 0, I1 = I1, I2 = I2, electronic_state=ElectronicState.B, 
                                        P = (-1)**(J_X+1), Omega = 1)
            
            #Calculate transition frequency
            frq = calculate_transition_frequency(X_state, B_state, H_tot_203, QN_203)/(1e9*4) #In GHz at IR
            
            #Print
            print(f"Frequency for T{J_X} F1' = {F1}, F' = {F} is f = {frq} GHz")
            T_transitions_203.append(f"T{J_X} F1' = {F1}, F' = {F} (Tl-203)")
            T_frequencies_203.append(frq)

## Dataframe of transitions
Generating a dataframe of all transitions and their frequencies

In [ ]:
import pandas as pd
pd.set_option("display.precision", 9)

In [ ]:
transitions = (Q_transitions + R_transitions + S_transitions + O_transitions + P_transitions + T_transitions
               + Q_transitions_203 + R_transitions_203 + S_transitions_203 + O_transitions_203 + P_transitions_203 
               + T_transitions_203)

frequencies = (Q_frequencies + R_frequencies + S_frequencies + O_frequencies + P_frequencies + T_frequencies + 
               Q_frequencies_203 + R_frequencies_203 + S_frequencies_203 + O_frequencies_203 + P_frequencies_203
               + T_frequencies_203)

df = pd.DataFrame([transitions, frequencies]).T
df.rename(columns = {0:'Transition', 1: "Frequency (GHz IR)"}, inplace = True)

In [ ]:
def gaussian_profile(center):
    sigma =  0.003 # Sigma in GHz
    x = center+np.linspace(-5,5,100)*sigma
    return x, np.exp(-(x-center)**2/(2*sigma**2))

In [ ]:
def plot_nearby_transitions(df, transition_name, lim = 1):
    # Fetch the transition and its frequency
    freq = df["Frequency (GHz IR)"][df["Transition"] == transition_name]
    if len(freq) == 0:
        print("Transition not found in dataframe")
        
    freq = freq.values[0]
        
    # Find transitions that are within lim from the named transition
    df_nearby = df[(np.abs(df["Frequency (GHz IR)"]-freq) < lim)]
    transitions = df_nearby["Transition"].values
    frequencies = df_nearby["Frequency (GHz IR)"].values
    print(df_nearby.sort_values(by = ["Frequency (GHz IR)"]))
        
    # Plot all the frequencies and their names
    fig, ax = plt.subplots()
#     ax.set_xlim(left = freq-lim, right = freq+lim)
    ax.set_title(transition_name+' and nearby transitions')
    ax.set_xlabel("Frequency (GHz IR)")
    
    for transition, frequency in zip(transitions, frequencies):
        if transition == transition_name:
            x, y = gaussian_profile(frequency)
            ax.plot(x,y,'tab:blue')
        
#         elif 'Tl-203' in transition:
#             if transition[0] in ['R','P']:
#                 x, y = gaussian_profile(frequency)
#                 ax.plot(x,y*0.25,'-', c = "tab:orange")

#             elif transition[0] == 'S':
#                 x, y = gaussian_profile(frequency)
#                 ax.plot(x,0.5*y*0.25,'-', c = "tab:orange")

#             elif transition[0] == 'T':
#                 x, y = gaussian_profile(frequency)
#                 ax.plot(x,0.1*y*0.25,'-', c = "tab:orange")
#             pass
                
        else:
            if transition[0] in ['R','P','Q']:
                x, y = gaussian_profile(frequency)
                ax.plot(x,y,'k')

#             elif transition[0] == 'S':
#                 x, y = gaussian_profile(frequency)
#                 ax.plot(x,0.5*y,'--', c = "tab:red")

#             elif transition[0] == 'T':
#                 x, y = gaussian_profile(frequency)
#                 ax.plot(x,0.1*y,'--', c = "tab:red")

In [ ]:
plot_nearby_transitions(df, "R2 F1' = 3.5, F' = 4.0", lim = 1000)

In [ ]:
df[(275850<df["Frequency (GHz IR)"]) & (275865>df["Frequency (GHz IR)"])].sort_values(by = "Frequency (GHz IR)")